In [1]:
drive = False  # False for Local
if drive:
    !pip install pretty_midi

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pretty_midi
from mido import Message, MidiFile, MidiTrack
import pandas as pd
import json
import torch as torch
import math
from tqdm import tqdm

In [3]:
if drive:
    from google.colab import drive
    drive.mount('/content/drive')
    my_drive_path = '/content/drive/MyDrive/2022_Special_Studies_Hablutzel/ModelCopies/'
else: # local
    my_drive_path = "./"

In [4]:
my_data = pd.read_csv(my_drive_path + 'maestro-v3.0.0/maestro-v3.0.0.csv')

In [5]:
my_data.iloc[45]

canonical_composer                                       Claude Debussy
canonical_title                          "Voiles" from Preludes, Book I
split                                                             train
year                                                               2008
midi_filename         2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...
audio_filename        2008/MIDI-Unprocessed_07_R3_2008_01-05_ORIG_MI...
duration                                                     212.295461
Name: 45, dtype: object

## Old Methods for Processing Pieces
- Sophia's uniform size method (400 beats each)
- Truncating/padding by fixed amount

In [6]:
def cut_piece_into_uniform_sizes(roll, split):
    """Sophia's method: cut pieces into uniform 400 beats
    NOT IN USE."""
    test = []
    train = []
    validation = []
    sev_roll = [] # store pieces
    target_size = 400 # how many beats/note onsets to use
    repeat_if_too_short = True # allow repeating the piece until target size, if the piece is too short
    use_only_first_chunk = False  # just keep first chunk of the piece
    
    # separate roll into pieces of target_size each
    if roll.shape[0]<target_size:
      if repeat_if_too_short:
        #if the piece is shorter than the target size, repeat the piece until it reaches the target size.
        while roll.shape[0]<target_size:
          roll = np.vstack((roll,roll))
        roll = roll[:target_size]
        sev_roll.append(np.array([roll,tempo,beats], dtype="object"))
      else:
        return []  # skip pieces less than target size
    else:
      if use_only_first_chunk:
        # just keep the first chunk of the piece
        num_arrays = 1
        leftover = 0
      else:
        # otherwise, divide up the piece into target-sized smaller pieces. If the remainder are longer than half the target size, make that a new piece as well.
        num_arrays = roll.shape[0]//target_size  # target_size pieces
        leftover = roll.shape[0]%target_size  # leftover beyond target_size pieces

      # get target_size pieces
      for l in range(0, num_arrays):
        start = target_size*l
        end = target_size*(l+1)
        sev_roll.append(np.array([roll[start:end],tempo,beats], dtype="object"))

      # if leftover larger than half the target size, repeat and use as well
      if leftover >= target_size//2:
        if repeat_if_too_short:
          leftover_roll = roll[end:]
          while leftover_roll.shape[0]<target_size:
            leftover_roll = np.vstack((leftover_roll,leftover_roll))
          leftover_roll = leftover_roll[:target_size]
          sev_roll.append(np.array([leftover_roll,tempo,beats], dtype="object"))
            
    # for each roll from this piece
    for roll in sev_roll:
    # add this into the correct file.
        add_roll_to_split(roll[0], roll[1], roll[2], split) # roll, tempo, beats, split
            
    return

In [7]:
def round_by_constant_interval(roll):
    """Round roll lengths by constant amount.
    NOT IN USE."""
    round_interval = 30
    round_up = True
    round_down = False
    
    # adds silence at the end of piece, so length is multiple of round_interval
    if round_up:
        beats_to_add = (roll.shape[0] * -1) % round_interval  # example, -4 mod 10 gives 6 beats to add
        pad = np.zeros((beats_to_add, 128))
        roll = np.vstack([roll, pad])
    # remove last few beats so length is multiple of round_interval
    elif round_down:
        beats_to_remove = roll.shape[0] % round_interval
        if beats_to_remove > 0:
            roll = roll[:(-1) * beats_to_remove] # cut off last [beats_to_remove] beats
    # else give piece in its exact size (with silence removed)
    
    return roll
    

## Initial Processing of Pieces
- convert to piano roll
- get num beats; tempo
- remove silence at beginning/end

In [8]:
path_beginning = my_drive_path + "maestro-v3.0.0/"
pieces = []

# how many steps you get
for i in tqdm(range(0, my_data.shape[0])):  # 1276 pieces
  # loads the piece into pretty_midi form
  path_part = my_data.iloc[i]["midi_filename"]
  split = my_data.iloc[i]["split"]
  path = path_beginning + path_part
  midi_data = pretty_midi.PrettyMIDI(path)

  # estimates the tempo of the piece
  tempo = midi_data.estimate_tempo()
  print(" estimated tempo:", tempo, end="")
  # gets piano roll sampled at estimated tempo
  # (diff tempo for each piece)
  roll = midi_data.get_piano_roll(tempo/60).T
  print("\tbeats:", roll.shape[0], end=" -> ")  # before silence removed or any other cut/padding
    
  # removes silence at beginning of piece
  over = False
  r = np.zeros(128)
  while over == False:
    m = roll[0]
    if np.array_equal(m,r):
      roll = roll[1:]
    else:
      over = True
    
  # removes silence at end of piece
  over = False
  r = np.zeros(128)
  while over == False:
    m = roll[-1]
    if np.array_equal(m,r):
      roll = roll[:-1]
    else:
      over = True
    
  # converts roll to binary matrix (only 0s and 1s)
  roll = np.where(roll > 0, 1, 0)
    
  # recalculate num beats after removing silence etc.
  beats = roll.shape[0]
  print(beats)  # ends print line from above (pre-processing beats)

  # save to data_sizes - beats, tempo, split, etc. for each piece
  this_piece = my_data.iloc[i].copy()
  this_piece["tempo"] = tempo
  this_piece["beats"] = beats
  this_piece["roll"] = roll
    
  pieces.append(this_piece)

  0%|          | 1/1276 [00:00<07:19,  2.90it/s]

 estimated tempo: 187.51378314519036	beats: 2200 -> 2194


  0%|          | 2/1276 [00:00<07:22,  2.88it/s]

 estimated tempo: 166.18828503917456	beats: 2118 -> 2113


  0%|          | 3/1276 [00:00<06:44,  3.14it/s]

...

 estimated tempo: 191.81988742963435	beats: 3642 -> 3638


100%|█████████▉| 1275/1276 [06:13<00:00,  3.20it/s]

 estimated tempo: 196.33873544853319	beats: 3498 -> 3493


100%|██████████| 1276/1276 [06:13<00:00,  3.41it/s]

 estimated tempo: 212.26533166457037	beats: 2231 -> 2226


## Use Piece Lengths to determine bin boundaries
(for rounding/truncating)

In [9]:
# sort pieces by beat length
pieces.sort(key=lambda x: x["beats"])

In [10]:
pieces[:2]

[canonical_composer                                Johann Sebastian Bach
 canonical_title           Prelude and Fugue in G Minor, WTC II, BWV 885
 split                                                             train
 year                                                               2011
 midi_filename         2011/MIDI-Unprocessed_22_R1_2011_MID--AUDIO_R1...
 audio_filename        2011/MIDI-Unprocessed_22_R1_2011_MID--AUDIO_R1...
 duration                                                       45.16956
 tempo                                                        139.989873
 beats                                                               103
 roll                  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
 Name: 840, dtype: object,
 canonical_composer                                          Franz Liszt
 canonical_title                  Transcendental Etude No. 1, “Preludio"
 split                                                        validation
 year                   

In [11]:
### find stats based on piece lengths
# sort piece lengths
all_beat_lengths = [piece["beats"] for piece in pieces]
all_beat_lengths.sort()  # should already be sorted, but just in case...
# print(all_beat_lengths[:10])

# ANOTHER OPTION: fit to percentiles
# oneth_percentile = all_beat_lengths[len(all_beat_lengths) // 100] # 1% of pieces are shorter 
# ninety_ninth_percentile = all_beat_lengths[(len(all_beat_lengths) * 99) // 100] # 1% of pieces are longer

# change bins, pl, and pu to change the logarithm
bins = 20
pl = all_beat_lengths[0] # fit to smallest piece    # or oneth_percentile
pu = all_beat_lengths[-1] # fit to largest piece    # or ninety_ninth_percentile

# logarithm is f(x) = a*e^(b*x)
a = pl * (pl / pu)**(1/(bins-1))
b = (1/(bins-1)) * math.log(pu/pl)

In [12]:
def update_bin_bounds(curr_bin, verbose=True):
    """calculates bounds for a given bin number"""
    lower_bound = int(a*math.exp(b * curr_bin))
    inner_bound = int(a*math.exp(b * (curr_bin+0.5)))
    upper_bound = int(a*math.exp(b * (curr_bin+1)))
    
    if verbose:
        print(f"current bin bounds for bin {curr_bin}: [{lower_bound}, {inner_bound}, {upper_bound}]")

    return lower_bound, inner_bound, upper_bound

In [13]:
def truncate(roll, beats, new_size):
    beats_to_remove = beats - new_size # shape aka beats
    if beats_to_remove > 0:
        roll = roll[:(-1) * beats_to_remove] # cut off last [beats_to_remove] beats
    return roll

def add_beats(roll, beats, new_size):
    beats_to_add = new_size - beats
    if beats_to_add > 0:  # never ever adding more than 1 repeat
        roll = np.vstack((roll,roll))
        roll = roll[:new_size] 
    return roll
    

In [14]:
def add_roll_to_split(roll, tempo, beats, split):
    # convert roll to tensor
    # and save roll, tempo, beats
    roll_data = np.array([torch.tensor(roll), tempo, beats], dtype="object")
    if split=="test":
        test.append(roll_data)
    elif split=="train":
        train.append(roll_data)
    else:
        validation.append(roll_data)
        
# clear lists each time data are added to rolls
train = []
test = []
validation = []

In [15]:
new_data_sizes = []

# process rolls
curr_bin = 1
lower_bound, inner_bound, upper_bound = update_bin_bounds(curr_bin)

# note: pieces are SORTED by beats, smallest to largest
for piece in pieces:
  roll = piece["roll"]
  split = piece["split"]
  beats = piece["beats"]
    
  # move to larger bin size
  if beats > upper_bound:
    curr_bin += 1  # update bounds
    lower_bound, inner_bound, upper_bound = update_bin_bounds(curr_bin)
    
  # piece is too small for smallest bin
  elif beats < lower_bound:  # shouldn't occur when smallest piece is used to fit the logarithm
    print(f"excluding beats {beats} as less than lower bound {lower_bound}")
    continue
    
  # fit piece to one of the bin bounds
  if beats > lower_bound and beats <= inner_bound: # truncate
    roll = truncate(roll, beats, lower_bound)
  elif beats > inner_bound and beats <= upper_bound: # round up by repeating
    roll = add_beats(roll, beats, upper_bound)
    
  # save roll
  add_roll_to_split(roll, tempo, roll.shape[0], split)  # shape for beats
    
  piece["new_beats"] = roll.shape[0]
  new_data_sizes.append(piece)
    
  ############
  # alternate ways to alter the size of the roll
  ############
    
  # Sophia's method of cutting into uniform chunks
  # NOT IN USE
  # cut_piece_into_uniform_sizes(roll, split)

  # save whole piece
  # add_roll_to_split(roll, split)  

current bin bounds for bin 1: [103, 115, 130]
current bin bounds for bin 2: [130, 146, 165]
current bin bounds for bin 3: [165, 185, 209]
current bin bounds for bin 4: [209, 235, 264]
current bin bounds for bin 5: [264, 298, 335]
current bin bounds for bin 6: [335, 377, 424]
current bin bounds for bin 7: [424, 478, 538]
current bin bounds for bin 8: [538, 605, 681]
current bin bounds for bin 9: [681, 766, 862]
current bin bounds for bin 10: [862, 971, 1092]
current bin bounds for bin 11: [1092, 1229, 1383]
current bin bounds for bin 12: [1383, 1557, 1752]
current bin bounds for bin 13: [1752, 1972, 2219]
current bin bounds for bin 14: [2219, 2497, 2810]
current bin bounds for bin 15: [2810, 3163, 3559]
current bin bounds for bin 16: [3559, 4005, 4508]
current bin bounds for bin 17: [4508, 5073, 5709]
current bin bounds for bin 18: [5709, 6424, 7229]
current bin bounds for bin 19: [7229, 8136, 9156]


In [16]:
# save new data sizes
data_sizes_df_new = pd.DataFrame.from_records(new_data_sizes)
data_sizes_df_new.to_csv(my_drive_path + "usable_data/data_sizes_post.csv")
# data_sizes_df_new.head()

In [17]:
# print bin sizes
data_sizes_df_new["new_beats"].value_counts()

2219    177
862     154
1092    153
1752    151
1383    135
681      97
2810     88
3559     62
538      58
424      53
4508     47
5709     47
7229     19
335      13
264      10
9156      8
165       2
209       1
103       1
Name: new_beats, dtype: int64

In [18]:
# see how many pieces are in each set
print("validation:\t", len(validation))
print("test:\t", len(test))
print("train:\t", len(train))

validation:	 137
test:	 177
train:	 962


In [19]:
# the data does not save until you run this.
torch.save(validation, my_drive_path + "usable_data/variable_bin_bounds_val.csv")
torch.save(test, my_drive_path + "usable_data/variable_bin_bounds_test.csv")
torch.save(train, my_drive_path + "usable_data/variable_bin_bounds_train.csv")